In [3]:
# import dependencies
import pandas as pd
from config import YT_API
import json
import requests
from googleapiclient.discovery import build
import matplotlib.pyplot as plt
import plotly.express as px
import numpy as np

In [27]:
#enter video id
video_id="t6fIp7mMJ90,et3u1a5RPdk,QMxl7lG0Ycg,kxsgBBKwP8E,CSiLPkUopBg"

In [28]:
def get_video_metadata(video_id):
    # creating youtube resource object
    youtube = build('youtube', 'v3', developerKey=YT_API)

    # get video title
    video_title = youtube.videos().list(part='snippet', id=video_id).execute()['items'][0]['snippet']['title']
    
    # retrieve youtube video results
    video_response = youtube.videos().list(part='snippet,contentDetails,statistics', id=video_id).execute()

    # create empty dataframe
    df = pd.DataFrame(columns=['video_id', 'channel_id', 'date_published','video_title', 'view_count ', 'like_count','comment_count'])

    # iterate video response
    while video_response:

    # export comments to a dataframe
        for i in range(len(video_response['items'])):
            #replies = []
            id = video_response['items'][i]['id']
            channel = video_response['items'][i]['snippet']['channelId']
            date= video_response['items'][i]['snippet']['publishedAt']
            title= video_response['items'][i]['snippet']['title']
            views= video_response['items'][i]['statistics']['viewCount']
            likes= video_response['items'][i]['statistics']['likeCount']
            comments= video_response['items'][i]['statistics']['commentCount']
            
            
            df = df.append({
                'video_id': id, 
                'channel_id': channel, 
                'date_published': date,
                'video_title':title, 
                'view_count':views,
                'like_count': likes, 
                'comment_count': comments 
                }, 
                ignore_index=True)
        if 'nextPageToken' in video_response:
            video_response = youtube.video().list(
            part='snippet,contentDetails,statistics',
            videoId=video_id,
            pageToken=video_response['nextPageToken']
            ).execute()
        else:
            break

    # change data types
    df['view_count'] = df['view_count'].astype(int)
    df['like_count'] = df['like_count'].astype(int)
    df['comment_count'] = df['comment_count'].astype(int)

    return df, video_title

In [24]:
df, video_title = get_video_metadata(video_id)

In [39]:
df=df.drop(df.columns[[4]], axis=1)
df


,video_id,channel_id,date_published,video_title,like_count,comment_count,view_count
0,9bZkp7q19f0,UCrDkAvwZum-UTjHmzDI2iIw,2012-07-15T07:46:32Z,PSY - GANGNAM STYLE(강남스타일) M/V,25860244,5332065,4582751936
1,CevxZvSJLk8,UC-8Q-hLdECwQmaWNwXitYDw,2013-09-05T20:00:22Z,Katy Perry - Roar (Official),15586013,760708,3673686440
2,KQ6zr6kCPj8,UCk78ZcA6kflEvBR0UrGDH0Q,2011-03-09T06:02:38Z,"LMFAO - Party Rock Anthem ft. Lauren Bennett, ...",10841926,648580,2223212377
3,pRpeEdMmmQ0,UCGnjeahCJW1AF34HBmQTJ-Q,2010-06-04T22:30:35Z,Shakira - Waka Waka (This Time for Africa) (Th...,18491759,1233403,3274175841
4,uelHwf8o7_U,UC20vb-R_px4CguHzzBPhoyQ,2010-08-05T19:09:46Z,Eminem - Love The Way You Lie ft. Rihanna,12441625,597028,2488042269


In [40]:
#save data to csv file- kaggle data
df.to_csv("music_video_metedata.csv")

In [30]:
df1, video_title = get_video_metadata(video_id)

In [31]:
df1

,video_id,channel_id,date_published,video_title,view_count,like_count,comment_count,view_count
0,t6fIp7mMJ90,UCpi8TJfiA4lKGkaXs__YdBA,2022-10-04T00:30:15Z,what happened.,NaN,950019,70621,11074447
1,et3u1a5RPdk,UCaXkIU1QidjPwiAYu6GcHjg,2022-10-24T17:34:00Z,Justice Thomas Temporarily Blocks Graham Testi...,NaN,2731,3195,305879
2,QMxl7lG0Ycg,UC8p1vwvWtl6T73JiExfWs1g,2022-10-24T17:33:19Z,Rishi Sunak to make history as U.K.'s next pri...,NaN,1186,1163,111859
3,kxsgBBKwP8E,UCxMXl9L79X1Mq_4iwCOyPFg,2022-10-22T23:33:15Z,Shooter in custody after allegedly killing 2 a...,NaN,426,422,48469
4,CSiLPkUopBg,UCPl2RYo0lLLqj8RaEADYclg,2022-06-10T04:00:05Z,Snarky Puppy - Trinity (Extended Version) (Emp...,NaN,34551,1880,1107138


In [41]:
#save data to csv file- api data
df1.to_csv("api_videos_metadata.csv")

Add video Id to original dataset for join later


In [36]:
#test on psy music video 
psy_id='9bZkp7q19f0'

psy_file=pd.read_csv("../YouTube-Spam-Collection-v1/Youtube01-Psy.csv")
psy_file["video_id"]=psy_id

#save file
psy_file.to_csv("musicvideo_id_files/psy.csv")
psy_file

,COMMENT_ID,AUTHOR,DATE,CONTENT,CLASS,video_id
0,LZQPQhLyRh80UYxNuaDWhIGQYNQ96IuCg-AYWqNPjpU,Julius NM,2013-11-07T06:20:48,"Huh, anyway check out this you[tube] channel: ...",1,9bZkp7q19f0
1,LZQPQhLyRh_C2cTtd9MvFRJedxydaVW-2sNg5Diuo4A,adam riyati,2013-11-07T12:37:15,Hey guys check out my new channel and our firs...,1,9bZkp7q19f0
2,LZQPQhLyRh9MSZYnf8djyk0gEF9BHDPYrrK-qCczIY8,Evgeny Murashkin,2013-11-08T17:34:21,just for test I have to say murdev.com,1,9bZkp7q19f0
3,z13jhp0bxqncu512g22wvzkasxmvvzjaz04,ElNino Melendez,2013-11-09T08:28:43,me shaking my sexy ass on my channel enjoy ^_^ ﻿,1,9bZkp7q19f0
4,z13fwbwp1oujthgqj04chlngpvzmtt3r3dw,GsMega,2013-11-10T16:05:38,watch?v=vtaRGgvGtWQ Check this out .﻿,1,9bZkp7q19f0
...,...,...,...,...,...,...
345,z13th1q4yzihf1bll23qxzpjeujterydj,Carmen Racasanu,2014-11-14T13:27:52,How can this have 2 billion views when there's...,0,9bZkp7q19f0
346,z13fcn1wfpb5e51xe04chdxakpzgchyaxzo0k,diego mogrovejo,2014-11-14T13:28:08,I don't now why I'm watching this in 2014﻿,0,9bZkp7q19f0
347,z130zd5b3titudkoe04ccbeohojxuzppvbg,BlueYetiPlayz -Call Of Duty and More,2015-05-23T13:04:32,subscribe to me for call of duty vids and give...,1,9bZkp7q19f0
348,z12he50arvrkivl5u04cctawgxzkjfsjcc4,Photo Editor,2015-06-05T14:14:48,hi guys please my android photo editor downloa...,1,9bZkp7q19f0


In [48]:
#test on katy music video 
katy_id='CevxZvSJLk8'

katy_file=pd.read_csv("../YouTube-Spam-Collection-v1/Youtube02-KatyPerry.csv")
katy_file["video_id"]=katy_id

#drop row 31- causing problems in sql database
katy_file=katy_file.drop(labels=[31], axis=0)

#save file
katy_file.to_csv("musicvideo_id_files/katy.csv")
katy_file

,COMMENT_ID,AUTHOR,DATE,CONTENT,CLASS,video_id
0,z12pgdhovmrktzm3i23es5d5junftft3f,lekanaVEVO1,2014-07-22T15:27:50,i love this so much. AND also I Generate Free ...,1,CevxZvSJLk8
1,z13yx345uxepetggz04ci5rjcxeohzlrtf4,Pyunghee,2014-07-27T01:57:16,http://www.billboard.com/articles/columns/pop-...,1,CevxZvSJLk8
2,z12lsjvi3wa5x1vwh04cibeaqnzrevxajw00k,Erica Ross,2014-07-27T02:51:43,Hey guys! Please join me in my fight to help a...,1,CevxZvSJLk8
3,z13jcjuovxbwfr0ge04cev2ipsjdfdurwck,Aviel Haimov,2014-08-01T12:27:48,http://psnboss.com/?ref=2tGgp3pV6L this is the...,1,CevxZvSJLk8
4,z13qybua2yfydzxzj04cgfpqdt2syfx53ms0k,John Bello,2014-08-01T21:04:03,Hey everyone. Watch this trailer!!!!!!!! http...,1,CevxZvSJLk8
...,...,...,...,...,...,...
345,z12sjp3zgtqnvlysj23zuxxaolrvd1oj504,Kacy Cluley,2015-06-05T18:59:52,This song means so much to me thank you soooo...,0,CevxZvSJLk8
346,z132enrpoy35yxpoe04cjr4zur3jvbyq3xo0k,Kasia Fabisiewicz,2015-06-05T19:02:05,&lt;3﻿,0,CevxZvSJLk8
347,z132jbmxfqm4fjysg23nwjfb2mv2vxnua,Decio Alves Martins,2015-06-05T19:29:20,"KATY PERRY, I AM THE ""DÉCIO CABELO"", ""DECIO HA...",1,CevxZvSJLk8
348,z12cdlswetvnejcri04cex0jfwy2u3tzj54,Rafi Hossain,2015-06-05T19:55:08,Honestly speaking except taylor swift and adel...,0,CevxZvSJLk8


In [46]:
#lmfao music video 
lmfao_id='KQ6zr6kCPj8'

lmfao_file=pd.read_csv("../YouTube-Spam-Collection-v1/Youtube03-LMFAO.csv")
lmfao_file["video_id"]=lmfao_id

#save file
lmfao_file.to_csv("musicvideo_id_files/lmfao.csv")
lmfao_file

,COMMENT_ID,AUTHOR,DATE,CONTENT,CLASS,video_id
0,z13uwn2heqndtr5g304ccv5j5kqqzxjadmc0k,Corey Wilson,2015-05-28T21:39:52.376000,"<a href=""http://www.youtube.com/watch?v=KQ6zr6...",0,KQ6zr6kCPj8
1,z124jvczaz3dxhnbc04cffk43oiugj25yzo0k,Epic Gaming,2015-05-28T20:07:20.610000,wierd but funny﻿,0,KQ6zr6kCPj8
2,z13tczjy5xj0vjmu5231unho1ofey5zdk,LaS Music,2015-05-28T19:23:35.355000,"Hey guys, I&#39;m a human.<br /><br /><br />Bu...",1,KQ6zr6kCPj8
3,z13tzr0hdpnayhqqc04cd3zqqqjkf3ngckk0k,Cheryl Fox,2015-05-28T17:49:35.294000,Party Rock....lol...who wants to shuffle!!!﻿,0,KQ6zr6kCPj8
4,z12pcvix4zedcjvyb04ccr1r0mr2g5xwyng0k,PATRICK_TW,2015-05-28T16:28:26.818000,Party rock﻿,0,KQ6zr6kCPj8
...,...,...,...,...,...,...
433,z13lvr4iupatjlrem231yvpxolzvspwdl,Salty Croc,2014-07-22T04:20:37.489000,Like this comment for no reason﻿,1,KQ6zr6kCPj8
434,z12lxhrqdkyusbkji04cihtrvn3jvxnqszg0k,Bob Orton,2014-07-22T00:26:50.820000,love this song﻿,0,KQ6zr6kCPj8
435,z12xhdjrsxm3v550w22oynsjrnmvjhkvj,LuckyMusiqLive,2014-07-21T22:25:54.048000,this song is awesome. these guys are the best....,1,KQ6zr6kCPj8
436,z13msngo3qvwx1ym223pehqgouexzdmnm,xXxPWND 420xXx,2014-07-21T11:05:51.945000,HOW MANY THUMBS UP FOR LOUIS SAVING THE DAY!?!?﻿,1,KQ6zr6kCPj8


In [45]:
#shakira music video 
shakira_id='pRpeEdMmmQ0'

shakira_file=pd.read_csv("../YouTube-Spam-Collection-v1/Youtube05-Shakira.csv")
shakira_file["video_id"]=shakira_id

#save file
shakira_file.to_csv("musicvideo_id_files/shakira.csv")
shakira_file

,COMMENT_ID,AUTHOR,DATE,CONTENT,CLASS,video_id
0,z13lgffb5w3ddx1ul22qy1wxspy5cpkz504,dharma pal,2015-05-29T02:30:18.971000,Nice song﻿,0,pRpeEdMmmQ0
1,z123dbgb0mqjfxbtz22ucjc5jvzcv3ykj,Tiza Arellano,2015-05-29T00:14:48.748000,I love song ﻿,0,pRpeEdMmmQ0
2,z12quxxp2vutflkxv04cihggzt2azl34pms0k,Prìñçeśś Âliś Łøvê Dømíñø Mâđiś™ ﻿,2015-05-28T21:00:08.607000,I love song ﻿,0,pRpeEdMmmQ0
3,z12icv3ysqvlwth2c23eddlykyqut5z1h,Eric Gonzalez,2015-05-28T20:47:12.193000,"860,000,000 lets make it first female to reach...",0,pRpeEdMmmQ0
4,z133stly3kete3tly22petvwdpmghrlli,Analena López,2015-05-28T17:08:29.827000,shakira is best for worldcup﻿,0,pRpeEdMmmQ0
...,...,...,...,...,...,...
365,_2viQ_Qnc6-bMSjqyL1NKj57ROicCSJV5SwTrw-RFFA,Katie Mettam,2013-07-13T13:27:39.441000,I love this song because we sing it at Camp al...,0,pRpeEdMmmQ0
366,_2viQ_Qnc6-pY-1yR6K2FhmC5i48-WuNx5CumlHLDAI,Sabina Pearson-Smith,2013-07-13T13:14:30.021000,I love this song for two reasons: 1.it is abou...,0,pRpeEdMmmQ0
367,_2viQ_Qnc6_k_n_Bse9zVhJP8tJReZpo8uM2uZfnzDs,jeffrey jules,2013-07-13T12:09:31.188000,wow,0,pRpeEdMmmQ0
368,_2viQ_Qnc6_yBt8UGMWyg3vh0PulTqcqyQtdE7d4Fl0,Aishlin Maciel,2013-07-13T11:17:52.308000,Shakira u are so wiredo,0,pRpeEdMmmQ0


In [47]:
#eminem music video 
eminem_id='uelHwf8o7_U'

eminem_file=pd.read_csv("../YouTube-Spam-Collection-v1/Youtube04-Eminem.csv")
eminem_file["video_id"]=eminem_id

#save file
eminem_file.to_csv("musicvideo_id_files/eminem.csv")
eminem_file

,COMMENT_ID,AUTHOR,DATE,CONTENT,CLASS,video_id
0,z12rwfnyyrbsefonb232i5ehdxzkjzjs2,Lisa Wellas,NaN,+447935454150 lovely girl talk to me xxx﻿,1,uelHwf8o7_U
1,z130wpnwwnyuetxcn23xf5k5ynmkdpjrj04,jason graham,2015-05-29T02:26:10.652000,I always end up coming back to this song<br />﻿,0,uelHwf8o7_U
2,z13vsfqirtavjvu0t22ezrgzyorwxhpf3,Ajkal Khan,NaN,"my sister just received over 6,500 new <a rel=...",1,uelHwf8o7_U
3,z12wjzc4eprnvja4304cgbbizuved35wxcs,Dakota Taylor,2015-05-29T02:13:07.810000,Cool﻿,0,uelHwf8o7_U
4,z13xjfr42z3uxdz2223gx5rrzs3dt5hna,Jihad Naser,NaN,Hello I&#39;am from Palastine﻿,1,uelHwf8o7_U
...,...,...,...,...,...,...
443,LneaDw26bFu3RCmyrWyP9S6wh1h9dBv3X95g1HzKAb4,Dany PK,NaN,SUBSCRIBE TO MY CHANNEL X PLEASE!. SPARE,1,uelHwf8o7_U
444,LneaDw26bFsD65dtIvAEObWYIYnFTqQDKBek_Ypz3J8,SmexyFriedChicken,NaN,Check out my videos guy! :) Hope you guys had ...,1,uelHwf8o7_U
445,LneaDw26bFuvs-8oWkLpAFa6g3QHpWD8k7sbbMP3Bg8,The Guy That's Done Everything,NaN,3 yrs ago I had a health scare but thankfully ...,1,uelHwf8o7_U
446,z12hfp2wmyuqztkw504cgblyxtbsxjuzeow0k,Jesse Pinkman,2015-05-06T11:42:44.601000,Rihanna looks so beautiful with red hair ;)﻿,0,uelHwf8o7_U
